In [ ]:
import copy
import random

# ======================================================================================
# Config
# ======================================================================================

BOARD_SIZE = 3
EMPTY = 0
PLAYER_X = 1  # clovek
PLAYER_O = 2  # AI

# ======================================================================================
# Helper funkcie
# ======================================================================================

def print_board(board):
    # Vypise stav hry
    symbols = {0: ' ', 1: 'X', 2: 'O'}
    print("\n  0   1   2")
    for i in range(3):
        print(f"{i} {symbols[board[i][0]]} | {symbols[board[i][1]]} | {symbols[board[i][2]]} ")
        if i < 2:
            print("  -----------")

def check_winner(board):
    # kontroluje, ci niekto vyhral vystup: 1 (X), 2 (O), alebo 0 (nikto)
    # Kontrola riadkov
    for row in board:
        if row[0] == row[1] == row[2] != 0:
            return row[0]
    
    # kontrola stlpcov
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != 0:
            return board[0][col]
    
    # Kontrola diagonal
    if board[0][0] == board[1][1] == board[2][2] != 0:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != 0:
        return board[0][2]
    
    return 0

def is_board_full(board):
    # kontroluje, ci je doska plna
    for row in board:
        if 0 in row:
            return False
    return True

def get_available_moves(board):
    # vrati zoznam dostupnych tahov ako (row, col) tuples
    moves = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves

def is_game_over(board):
    # kontroluje, ci je hra ukoncena (vitaz alebo remiza)
    has_winner = check_winner(board) != 0
    is_full = is_board_full(board)
    
    if has_winner:
        return has_winner
    if is_full:
        return is_full
    return False

# ======================================================================================
# Minimax algoritmus
# ======================================================================================

def evaluate_board(board):
    """
    Hodnoti poziciu na boarde
    +10 ak vyhral AI (O)
    -10 ak vyhral clovek (X)  
    0 ak remiza alebo hra pokracuje
    """
    winner = check_winner(board)
    if winner == PLAYER_O:  # AI vyhralo
        return 10
    elif winner == PLAYER_X:  # clovek vyhral
        return -10
    else:
        return 0

def minimax(board, depth, is_maximizing, alpha=float('-inf'), beta=float('inf')):
    """
    Minimax algoritmus s alpha-beta pruning.
    
    Parametre:
        board: Aktuálna doska
        depth: Hĺbka v strome hľadania
        is_maximizing: True ak je na ťahu maximalizujúci hráč (AI)
        alpha: Alpha hodnota pre pruning
        beta: Beta hodnota pre pruning
    
    Návratová hodnota:
        Najlepšie skóre pre aktuálnu pozíciu
    """
    # kontrola ci niekto nevyhral
    if is_game_over(board):
        score = evaluate_board(board)
        
        # prisposobenie skore podla hlbky, preferujeme skorsie vyhry
        if score > 0:  # AI vyhrava
            adjusted_score = score - depth
            return adjusted_score
        elif score < 0:  # clovek vyhrava
            adjusted_score = score + depth
            return adjusted_score
        else:  # remiza
            return 0
    
    if is_maximizing:
        # AI sa snazi maximalizovat skore
        max_eval = float('-inf')
        available_moves = get_available_moves(board)
        
        for move in available_moves:
            row, col = move
            
            # urobime tah
            board[row][col] = PLAYER_O
            
            # rekurzivne volanie
            eval_score = minimax(board, depth + 1, False, alpha, beta)
            
            # vratime tah spat
            board[row][col] = EMPTY
            
            # aktualizujeme najlepsie skore
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            
            # alpha-beta pruning (orezavanie)
            if beta <= alpha:
                break
        
        return max_eval
    else:
        # clovek sa snazi minimalizovat skore
        min_eval = float('inf')
        available_moves = get_available_moves(board)
        
        for move in available_moves:
            row, col = move
            
            # urobime tah
            board[row][col] = PLAYER_X
            
            # rekurzivne volanie
            eval_score = minimax(board, depth + 1, True, alpha, beta)
            
            # Vratime tah spat
            board[row][col] = EMPTY
            
            # aktualizujeme najlepsie skore
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            
            # apha-beta pruning (orezavanie)
            if beta <= alpha:
                break
        
        return min_eval
    
def get_best_move(board):
    """
    Najde najlepsi tah pre AI pomocou minimax algoritmu
    
    Returns:
        (row, col) - suradnice najlepsieho tahu
    """
    best_move = None
    best_value = float('-inf')
    
    for row, col in get_available_moves(board):
        # urobime tah
        board[row][col] = PLAYER_O
        # vyhodnotime pomocou minimax
        move_value = minimax(board, 0, False)
        # vratime tah
        board[row][col] = EMPTY
        
        # ak je tento tah lepsi, zapamatajme si ho
        if move_value > best_value:
            best_value = move_value
            best_move = (row, col)
    
    return best_move

# ======================================================================================
# Herne funkcie
# ======================================================================================

def make_ai_move(board):
    # urobi tah AI
    move = get_best_move(board)
    if move:
        row, col = move
        board[row][col] = PLAYER_O
        print(f"AI hra na poziciu ({row}, {col})")
    return board

def make_human_move(board):
    # spracuje tah cloveka
    while True:
        try:
            print("\nTvoj tah (X):")
            row = int(input("Zadaj riadok (0-2): "))
            col = int(input("Zadaj stlpec (0-2): "))
            
            if row < 0 or row > 2 or col < 0 or col > 2:
                print("Neplatne suradnice! Zadaj cisla 0-2.")
                continue
            
            if board[row][col] != EMPTY:
                print("Policko je obsadene! Vyber ine.")
                continue
            
            board[row][col] = PLAYER_X
            break
            
        except ValueError:
            print("Neplatny vstup! Zadaj cisla.")
        except KeyboardInterrupt:
            print("\nKoniec hry.")
            exit()
    
    return board

def play_game():
    # hlavna herna funkcia / loop
    print("="*50)
    print("TIC-TAC-TOE s MINIMAX algoritmom")
    print("="*50)
    print("Ty si X, AI je O")
    print("Suradnice sa zadavaju ako riadok a stlpec (0-2)")
    
    # inicializacia boardu
    board = [[EMPTY for _ in range(3)] for _ in range(3)]
    
    # vyprintuje moznosti kto zacina
    print("\nKto ma zacat?")
    print("1 - Ty (X)")
    print("2 - AI (O)")
    
    while True:
        try:
            choice = int(input("Vyber (1 alebo 2): "))
            if choice in [1, 2]:
                current_player = PLAYER_X if choice == 1 else PLAYER_O
                break
            else:
                print("Zadaj 1 alebo 2.")
        except ValueError:
            print("Neplatny vstup!")
    
    # herny loop
    while True:
        print_board(board)
        
        if current_player == PLAYER_X:
            # tah cloveka
            board = make_human_move(board)
        else:
            # tah AI
            print("\nAI rozmysla...")
            board = make_ai_move(board)
        
        # kontrola vitaza
        winner = check_winner(board)
        if winner != 0:
            print_board(board)
            if winner == PLAYER_X:
                print("\nyhral si")
            else:
                print("\nAI vyhralo")
            break
        
        # kontrola remizy
        if is_board_full(board):
            print_board(board)
            print("\nRemiza")
            break
        
        # zmena hraca
        current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X

# ======================================================================================
# Testovacie funkcie
# ======================================================================================

def test_minimax():
    # testuje minimax algoritmus na predom definovanych poziciach
    print("Testovanie minimax algoritmu...")
    
    # Test 1: AI ma moznost vyhrat v jednom tahu
    test_board1 = [
        [1, 2, 0],
        [1, 2, 0], 
        [0, 0, 0]
    ]
    print("\nTest 1 - AI moze vyhrat:")
    print_board(test_board1)
    move = get_best_move(test_board1)
    print(f"Najlepsi tah: {move}")
    
    # Test 2: AI musi blokovat vitazstvo protivnika
    test_board2 = [
        [1, 1, 0],
        [2, 0, 0],
        [0, 0, 0]
    ]
    print("\nTest 2 - AI musi blokovat:")
    print_board(test_board2)
    move = get_best_move(test_board2)
    print(f"Najlepsi tah: {move}")

def benchmark_performance():
    # testuje vykonnost minimaxu proti nahodnym vyberom
    print("\nBenchmark - AI vs nahodný hráč (100 hier)...")
    
    ai_wins = 0
    draws = 0
    
    for game in range(100):
        board = [[EMPTY for _ in range(3)] for _ in range(3)]
        current_player = PLAYER_X if random.random() < 0.5 else PLAYER_O
        
        while True:
            if current_player == PLAYER_O:
                # AI tah
                move = get_best_move(board)
                if move:
                    board[move[0]][move[1]] = PLAYER_O
            else:
                # nahodny tah
                available = get_available_moves(board)
                if available:
                    move = random.choice(available)
                    board[move[0]][move[1]] = PLAYER_X
            
            winner = check_winner(board)
            if winner == PLAYER_O:
                ai_wins += 1
                break
            elif winner == PLAYER_X:
                break
            elif is_board_full(board):
                draws += 1
                break
            
            current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X
    
    print(f"Výsledky: AI vyhralo {ai_wins}/100, remizy: {draws}")
    print(f"Úspešnosť AI: {(ai_wins + draws)/100*100:.1f}%")

# ======================================================================================
# Main
# ======================================================================================

if __name__ == "__main__":
    while True:
        print("\n" + "="*50)
        print("MINIMAX TIC-TAC-TOE")
        print("="*50)
        print("1 - Hrať hru")
        print("2 - Testovať minimax")
        print("3 - Benchmark výkonnosti")
        print("4 - Ukončiť")
        
        try:
            choice = int(input("\nVyber možnosť (1-4): "))
            
            if choice == 1:
                play_game()
            elif choice == 2:
                test_minimax()
            elif choice == 3:
                benchmark_performance()
            elif choice == 4:
                print("Ďakujem za hru!")
                break
            else:
                print("Neplatná voľba!")
                
        except ValueError:
            print("Zadaj platné číslo!")
        except KeyboardInterrupt:
            print("\nKoniec programu.")
            break